# Using Tableau Methods

Runge-Kutta methods are defined by Butcher tableaus. These are matrices of coefficients which define the steps of a Runge-Kutta method. This simplified way of specifying Runge-Kutta methods is useful for ease of implementation and sharing of methods.

Note: These methods are generally for researchers or mathematicans who are interested in testing their own (or a large assortment of) methods. If you're simply looking for fast and optimized methods, this is not the right place!

## The General Tableau Solver

DifferentialEquations.jl has general tableau methods which allow a user to easily to define new numerical methods simply by defining the associated coefficients. These methods will utilize many performance optimizations, such as being fully cached and devectorized, and can designed as a FSAL scheme to skip a function calculation per step. Note that the tableau methods are not the most efficient: hand-optimized methods allow one to unroll the tableau into stack allocated coefficients which will always do better. That is why DifferentialEquations.jl offers optimized versions of the most popular solvers like DP5.

However, DifferentialEquations.jl offers over a hundred tableaus to play with. Please see [the developer documentation](http://devdocs.juliadiffeq.org/latest/internals/tableaus.html) for a full list. For example, we can construct the Dormand-Prince 4/5 tableau via the command:

In [1]:
using DifferentialEquations, DiffEqProblemLibrary, DiffEqDevTools
tab = constructDormandPrince()

INFO: Recompiling stale cache file /home/crackauc/.julia/lib/v0.6/DiffEqDevTools.ji for module DiffEqDevTools.
INFO: Recompiling stale cache file /home/crackauc/.julia/lib/v0.6/DifferentialEquations.ji for module DifferentialEquations.


DiffEqBase.ExplicitRKTableau{Array{Float64,2},Array{Float64,1},true}([0.0 0.0 … 0.0 0.0; 0.2 0.0 … 0.0 0.0; … ; 2.84628 -10.7576 … 0.0 0.0; 0.0911458 0.0 … 0.130952 0.0], [0.0, 0.2, 0.3, 0.8, 0.888889, 1.0, 1.0], [0.0911458, 0.0, 0.449236, 0.651042, -0.322376, 0.130952, 0.0], [0.0899132, 0.0, 0.453489, 0.614062, -0.271512, 0.0890476, 0.025], 7, 5, 4)

Here is what that function looks like:

In [ ]:
function constructDormandPrince(T::Type = Float64)
  A = [0 0 0 0 0 0 0
      1//5 0 0 0 0 0 0
      3//40 9//40 0 0 0 0 0
      44//45 -56//15 32//9 0 0 0 0
      19372//6561 -25360//2187 64448//6561 -212//729 0 0 0
      9017//3168 -355//33 46732//5247 49//176 -5103//18656 0 0
      35//384 0 500//1113 125//192 -2187//6784 11//84 0]
  c = [0;1//5;3//10;4//5;8//9;1;1]
  α = [35//384;0;500//1113;125//192;-2187//6784;11//84;0]
  αEEst = [5179//57600;0;7571//16695;393//640;-92097//339200;187//2100;1//40]
  return(ExplicitRKTableau(A,c,α,5,αEEst=αEEst,adaptiveorder=4,fsal=true))
end

It's pretty clear how this is done: A is the Butcher matrix, c are the timestep sizes, α are the end summation weights, and αEEst are the end summation weights for the error estimate. Then we construct the `ExplicitRKTableau` object using these arrays, plus we feed it the order, the order of the error estimating step, and whether or not the scheme uses FSAL (First Same as Last). 


DifferentialEquations.jl can solve directly using tableau using the `ExplicitRK` method with the keyword option `tableau`. For example:

In [3]:
sol = solve(prob_ode_linear,ExplicitRK(tableau=tab))
using Plots
gr()
plot(sol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.5 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,368.504 48.7388,364.937 58.1076,361.308 67.4763,357.617 76.8451,353.862 86.2138,350.042 95.5825,346.156 104.951,342.204 114.32,338.183 123.689,334.093 
 133.058,329.932 142.426,325.699 151.795,321.393 161.164,317.013 170.532,312.556 179.901,308.023 189.27,303.411 198.639,298.72 208.007,293.948 217.376,289.093 
 226.745,284.155 236.114,279.132 245.482,274.023 254.851,268.826 264.22,263.539 273.589,258.161 282.957,252.689 292.326,247.123 301.695,241.46 311.064,235.699 
 320.432,229.838 329.801,223.876 339.17,217.811 348.539,211.641 357.907,205.364 367.276,198.979 376.645,192.484 386.014,185.878 395.382,179.159 404.751,172.325 
 414.12,165.373 423.489,158.302 432.857,151.108 442.226,143.79 451.595,136.345 460.964,128.771 470.332,121.066 479.701,113.227 489.07,105.252 498.439,97.1381 
 507.807,88.8837 517.176,80.4862 526.545,71.9433 535.914,63.2526 545.282,54.4119 554.651,45.4187 564.02,36.2708 573.388,26.9659 582.757,17.5015 592.126,7.87402 
 
 "/>
 
 
 
 
 u1(t)

## Stability Region Plotting

Note that a plot recipe is defined for tableaus which allows one to plot the stability region. For example:

In [4]:
plot(tab)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
<image width="553" height="361" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAikAAAFpCAYAAABZI7jvAAAKrUlEQVR4nO3d0W7bxhaG0RGjPqcR
nOcqggB9ywryuVAT27FlS9SQ88/MWpdtahBBKX3ee0QdSvn2XAAAwiytLwAA4CMiBQCIJFIAgEgi
BQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCI
JFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIA
gEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIdHzplHPT
CwEAZvd2dnI4nf95Pi7fi0gBALZ3CZHT+ceXf/K49aUAALNbboqSP4kUAGAzp/PP1f+tSAEAqnsk
Tn7x6R4AoKoagVKKSAEAqqqXFiIFAKhmzQHZa0QKAFBJ3awQKQBAFTWnKKWIFACgkuPyVPXniRQA
oJqaoSJSAIBIIgUAqKrWNEWkAADVHZenh2PFY/EBgM38GSr3PI32UMq359oXBABwqz/D5bh8L6Wc
RQoAkMmZFAAgkkgBACKJFAAgkkgBACKJFAAgkkgBACKJFAAgkkgBACKJFAAgkkgBACL5gkGAbr3/
PfN0/lHtp/+1/K88l/OVf3vtn0M9vrsHINpLiNQMkJrex4yAoQ6RAtDcJURSI6SGy7fa/iJiuI1I
AdjN+DGyxkvAiBfeEikAm1jEyAPerpDEy6xECkA1JiVbECzzEikAq4mSFkTLPEQKwF2scdKIlnGJ
FIBPmZb0RLCMRaQAfMjEZAQ+OdQ3kQLwmzAZ1WXCcmp9GdxJpACIk2m8rINMVnogUoBpnc4/W18C
DV1WQWIlmUgBJmJiwnumK7lECjAJgcLnTFbyiBRgaFY63Eus5BApwKBMTniMWGlPpABDMTmhJudV
2lpaXwBAHYtAobp/z3+Xg7fKZkxSgK4JE/ZgotKGSAE65cwJ+3NOZV8iBeiKyQmtecT+fizagE44
c0KGf89/F2+f+zi2vgCAr4gT0hzKUi5rCKufLVn3AMGcOyGbMyrbEilAHJMTeiJUtmOpBgRx7oT+
mPZtx5kUIIC1Dr1bimlKfSYpQGMChf5dnkrrLbU2Z1KABoQJ4/FU2vpkH7AzgcKYLs9PoSaTFGAX
DsQyC5/2qcckBdiBlxrgfiYpwEasdZjXcXlqfQlD8OsNsAGBwuy8vdbgOSlAJcIEqEvqAVUIFKA2
Z1KAB5iewDU+5fM46x5gFR8pBrYmUoA7mJzArQ5lKZdVhWnKWs6kADcTKHA7T6B9nEkK8AXTE6AN
kQJc5dwJ0JJIAf5gcgK1XM6lOJOylo8gA7+ZnEB9HpG/noOzQLlMTwQKkMW6B6ZmtQPkMkmBaQkU
IJtJCkzGWgfohUkKTMUtD/tz361lkgJTsNoB+iNSYHDWO0CvRAoMSpwAvbMog+F45gkwBpECQ3H2
BBiHdQ8MQZwA4xEp0DmrHWBU1j3QMYECjEykQKcECjA66x7ojDgBZmGSAl1xywLzMEmBTpigALPx
axl0QKAAMxIpEM9tCszJugeCmaAAM/MrGkTy/TsAIgUCecQ9gEiBQG5LgFKcSYEoVjwAL/zKBiEE
CsBbIgUCCBSA90QKABBJpEBjpigAHxMp0JBAAbhOpEAjAgXgcyIFmnDrAXzFKyU04ImyMI/L/e7t
dg1/awBAJJECO3MWBeA2IgV25ZYDuJVXTNiRsygAtxMpsBNrHoD7iBQAIJJIAQAiiRTYhVsN4F5e
OWEHDswC3E+kwMYcmAVYR6QAAJFECgAQSaTAhqx6ANYTKQBAJJECm3F7ATzCqyhsxMeOAR4jUgCA
SCIFAIgkUgCASCIFAIgkUgCASCIFAIgkUgCASCIFAIgkUmADvrMH4HEiBQCIJFIAgEgiBQCIJFIA
gEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgi
BQCIJFIAgEgiBQCIJFIAgEgiBTZwXJ5aXwJA90QKABBJpAAAkUQKABBJpAAAkUQKABBJpAAAkUQK
ABBJpAAAkUQKABBJpMBGPHUW4DEiBQCIJFIAgEgiBTZk5QOwnkgBACKJFNiYaQrAOiIFAIgkUmAH
pikA9xMpAEAkkQIARBIpAEAkkQI7cS4F4D4iBXYkVABuJ1IAgEgiBXZmmgJwG5ECAEQSKdCAaQrM
47h8L6WcW19Gl0QKABBJpEAjpikAnxMp0JBQAbhOpEBjQgXgYyIFAIgkUiCAaQrAeyIFQhyXJ7EC
8IpIgTBCBeBCpAAAkUQKBDJNARApEMsZFWB2IgXCCRVgViIFOiBUgBmJFOiEUAFmI1KgI0IFmIlI
gc4IFWAWIgU65JM/wAxECnRMqAAjEynQOaECjEqkwACsf4ARiRQYiFABRiJSYDCmKsAoRAoMSqgA
vRMpMDChAvRMpMDgrH+AXokUmIRYgVbOrS+gWyIFJiNUgF6IFJiQqQrQA5ECExMrQDKRAogVIJJI
AX4TKkCSQynfnltfBJDndP7Z+hKge8fle/HpnvVMUoAPmaoArR1bXwCQ63WomKwAezNJAW5isgLs
zZkU4G6mKvA151EeZ5IC3M1UBdiDSQrwMJMVeMsUpQ6TFOBhJivAFkQKUIWn1gK1iRSgKrECpVj1
1CFSgE2IFeBRDs4Cu3C4llk4NFuPSQqwC1MVZiBQ6hIpwG6sgIB7WPcAzVgBMRJTlPpECtCcWKF3
AmUb1j1Ac9ZA9E+gbEGkADHECvCadQ8QyQqIHljzbEukANHECqkEyvZECtAFsUISgbIPZ1KALjiv
QhaBsgeRAnRFqNDaZYrCHqx7gK5ZA7EXK579maQAXTNZYQ+X/88Eyt5MUoChmKxQmwlKOyIFGI5Q
oQZx0p5IAYYmWFhDoGQQKcDwhAq3EidZRAowDbHCNeIkk0gBpiNW+EWcZBMpwNQEy5zESR9ECsB/
BMvYhEl/RArABwTLOMRJv0QKwBVCpW/ipH8iBeBGoiWfr0kYi0gBWEGwZBAlYxMpABWIln2IkrmI
FIANiJY6nCuZm0gB2JhguZ0o4TWRAtDA7OEiRriFSAEIM0rACBEeJVIAOtUqZsQHexEpAECkpfUF
AAB8RKQAAJFECgAQSaQAAJFECgAQSaQAAJFECgAQSaQAAJFECgAQSaQAAJFECgAQ6XA6//Psy6IA
gH0s5XT+cdOfPG58JQDA9G4Pk9dECgCwkXVx8otIAQCqOp1/Vvk5Ds4CAJFECgBQTa0pSikiBQAI
JVIAgCpqTlFKESkAQCXH5anqzxMpAEAkkQIAVFNzmiJSAIBIIgUAqOq4PFWZqIgUAGATj8aKSAEA
NrU2Vg6lfHve4HoAAL702bNVRAoAEMm6BwCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIA
gEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgi
BQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIAgEgiBQCIJFIA

We can modify the plot by changing the `xlim=[xmin,xmax]` and `ylim=[ymin,ymax]`, and change the accuracy of the plot via `Δx` and `Δy`. For example, let's plot the stability region of the implicit Euler method:

In [5]:
tab = constructImplicitEuler()
plot(tab,xlim=[-1,1],ylim=[-1,1],dx=1//20,dy=1//20)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
<image width="599" height="1803" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAlcAAAcLCAYAAACXT5avAAAgAElEQVR4nOzdPWpdWRaA0fuMJlGa
iRuHpcRW6iFoPEoNRpkCYwdtQaHIYMooKM3AmoUHID91UIaiwUfdvu/T+10r1OGiE37aHLYW98vL
hwkAgMSzTV8AAGCfiCsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsA
gJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4
AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAIiSsAgJC4AgAI
iSsAgJC4AgAIiSsAgJC4